In [27]:
from datasets import load_dataset
from transformers import BertTokenizer, BertForMaskedLM
from transformers import AdamW
import torch
from torch.utils.data import DataLoader, TensorDataset
import nltk

# Ensure you have the NLTK data required for edit distance
nltk.download('punkt')

dataset = load_dataset("HamdanXI/paradetox_with_editOps")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Function to calculate edit distance
def calculate_edit_distance(s1, s2):
    return nltk.edit_distance(s1, s2)

def preprocess_data(samples):
    processed_comments = []
    labels = []
    edit_distances = []  # To store edit distances

    for sample in samples:
        toxic_comment = sample['en_toxic_comment']
        neutral_comment = sample['en_neutral_comment']
        edit_ops = sample['edit_ops']
        words = toxic_comment.split()

        # Calculate and store the edit distance
        edit_distance = calculate_edit_distance(toxic_comment, neutral_comment)
        edit_distances.append(edit_distance)

        for operation in sorted(edit_ops, key=lambda op: int(op[2]), reverse=True):
            op_type, text, index = operation[:3]
            index = int(index)

            if op_type == "replace":
                words[index:index+len(text.split())] = ['[MASK]']
            elif op_type == "delete":
                del words[index:index+len(text.split())]
            elif op_type == "insert":
                words.insert(index, '[INSERT]')

        masked_comment = ' '.join(words)

        encoded_comment = tokenizer.encode_plus(
            masked_comment,
            max_length=35,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        label_ids = tokenizer.encode_plus(
            sample['en_neutral_comment'],
            max_length=197,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )['input_ids']

        processed_comments.append(encoded_comment)
        labels.append(label_ids)

    return processed_comments, labels, edit_distances

# Preprocess the dataset
processed_comments, labels, edit_distances = preprocess_data(dataset["train"])

from transformers import BertForMaskedLM

model = BertForMaskedLM.from_pretrained('bert-base-uncased')

import torch
from torch.utils.data import DataLoader, TensorDataset

input_ids = torch.cat([c['input_ids'] for c in processed_comments], dim=0)
attention_mask = torch.cat([c['attention_mask'] for c in processed_comments], dim=0)
labels_prepared = torch.cat(labels, dim=0).squeeze()

labels_prepared = labels_prepared[:, :input_ids.size(1)]

dataset_tensor = TensorDataset(input_ids, attention_mask, labels_prepared)
loader = DataLoader(dataset_tensor, batch_size=8, shuffle=True)

# Convert the edit distances to a tensor
edit_distances_tensor = torch.tensor(edit_distances).unsqueeze(1)  # Make it 2D for concatenation

# Add the edit distances tensor to your dataset
dataset_tensor = TensorDataset(input_ids, attention_mask, labels_prepared, edit_distances_tensor)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)
model.train()

for epoch in range(3):
    for batch in loader:
        b_input_ids, b_attention_mask, b_labels = batch
        b_input_ids = b_input_ids.to(device)
        b_attention_mask = b_attention_mask.to(device)
        b_labels = b_labels.to(device)

        outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        print(f"Loss: {loss.item()}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
C:\Users\User\anaconda3\lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a fut

Loss: 15.264782905578613
Loss: 11.179194450378418
Loss: 12.525819778442383
Loss: 9.474989891052246
Loss: 9.525128364562988
Loss: 9.054673194885254
Loss: 9.353623390197754
Loss: 8.493534088134766
Loss: 7.597638130187988
Loss: 7.354576110839844
Loss: 6.5820231437683105
Loss: 6.4792680740356445
Loss: 6.634530544281006
Loss: 5.552680969238281
Loss: 5.463038921356201
Loss: 6.871428489685059
Loss: 5.247280120849609
Loss: 6.261411190032959
Loss: 4.309406757354736
Loss: 4.699001312255859
Loss: 4.653511047363281
Loss: 4.591294288635254
Loss: 3.341404914855957
Loss: 1.93160879611969
Loss: 3.724277973175049
Loss: 3.841409683227539
Loss: 2.9184839725494385
Loss: 3.2299294471740723
Loss: 1.9740034341812134
Loss: 2.8911895751953125
Loss: 3.132412910461426
Loss: 2.2097764015197754
Loss: 2.517139196395874
Loss: 2.6614651679992676
Loss: 1.8161643743515015
Loss: 1.5112051963806152
Loss: 2.4038054943084717
Loss: 2.5557048320770264
Loss: 3.1077463626861572
Loss: 2.2638986110687256
Loss: 2.5043270587921143

Loss: 1.4498199224472046
Loss: 0.9843282699584961
Loss: 0.8056577444076538
Loss: 1.5398576259613037
Loss: 0.9676905870437622
Loss: 1.076363205909729
Loss: 1.1316226720809937
Loss: 0.8625157475471497
Loss: 0.9342125058174133
Loss: 1.1389048099517822
Loss: 1.1541597843170166
Loss: 0.871662974357605
Loss: 1.142496943473816
Loss: 1.4830074310302734
Loss: 0.5900209546089172
Loss: 1.0021440982818604
Loss: 1.049037218093872
Loss: 1.2863247394561768
Loss: 1.501983404159546
Loss: 0.8268482089042664
Loss: 1.3944249153137207
Loss: 0.5408893823623657
Loss: 1.210933804512024
Loss: 1.2712866067886353
Loss: 0.5129765272140503
Loss: 1.3724550008773804
Loss: 0.6864273548126221
Loss: 1.473718523979187
Loss: 1.5066026449203491
Loss: 0.9965586066246033
Loss: 1.0498788356781006
Loss: 1.0712971687316895
Loss: 1.7810168266296387
Loss: 0.9634482264518738
Loss: 1.373011589050293
Loss: 0.6285570859909058
Loss: 0.9842227101325989
Loss: 1.1794389486312866
Loss: 1.1665812730789185
Loss: 0.7559673190116882
Loss: 1.

Loss: 1.0422987937927246
Loss: 0.7111929059028625
Loss: 1.1543511152267456
Loss: 1.8526757955551147
Loss: 1.2781667709350586
Loss: 1.0403680801391602
Loss: 1.4329216480255127
Loss: 1.3312777280807495
Loss: 1.309647560119629
Loss: 0.973450779914856
Loss: 1.064260482788086
Loss: 0.8301237225532532
Loss: 0.8435379266738892
Loss: 0.6705092787742615
Loss: 0.641930103302002
Loss: 1.2064534425735474
Loss: 1.126468539237976
Loss: 1.6372919082641602
Loss: 1.892816424369812
Loss: 1.4541256427764893
Loss: 1.0840790271759033
Loss: 0.6881682872772217
Loss: 1.088209629058838
Loss: 0.8154242634773254
Loss: 1.380433440208435
Loss: 1.1927785873413086
Loss: 1.3708561658859253
Loss: 0.8121283054351807
Loss: 1.4458942413330078
Loss: 1.0132884979248047
Loss: 0.6985247731208801
Loss: 0.7249218821525574
Loss: 1.4185469150543213
Loss: 0.9295326471328735
Loss: 2.395214319229126
Loss: 1.254596471786499
Loss: 0.7544639110565186
Loss: 1.0055440664291382
Loss: 0.7321240901947021
Loss: 1.3581552505493164
Loss: 0.43

Loss: 1.525678038597107
Loss: 2.1775901317596436
Loss: 1.0135301351547241
Loss: 1.046983242034912
Loss: 1.1204862594604492
Loss: 1.013904094696045
Loss: 1.1777682304382324
Loss: 1.230007529258728
Loss: 1.3541792631149292
Loss: 0.5877089500427246
Loss: 0.9944273829460144
Loss: 1.052996039390564
Loss: 0.8092287182807922
Loss: 0.8095942735671997
Loss: 0.8723583817481995
Loss: 0.885127067565918
Loss: 0.9062250256538391
Loss: 1.6616569757461548
Loss: 0.49875378608703613
Loss: 0.8231703042984009
Loss: 1.3344241380691528
Loss: 0.9007482528686523
Loss: 1.2439380884170532
Loss: 0.7001857757568359
Loss: 1.1219037771224976
Loss: 0.9381638765335083
Loss: 1.2124894857406616
Loss: 0.8714573383331299
Loss: 1.318984031677246
Loss: 1.182458758354187
Loss: 1.1019058227539062
Loss: 1.3381235599517822
Loss: 0.9701216816902161
Loss: 1.5041838884353638
Loss: 0.9395325779914856
Loss: 0.9116942882537842
Loss: 1.0697910785675049
Loss: 0.7011708617210388
Loss: 1.0834800004959106
Loss: 0.8957828283309937
Loss: 0

...

Loss: 0.4446454346179962
Loss: 0.6681385636329651
Loss: 0.6364821195602417
Loss: 0.77383953332901
Loss: 1.0913454294204712
Loss: 0.47209134697914124
Loss: 0.8324313163757324
Loss: 0.9464174509048462
Loss: 0.6885817646980286
Loss: 0.7767326831817627
Loss: 0.5646944046020508
Loss: 0.6951138377189636
Loss: 0.7445853352546692
Loss: 0.8020858764648438
Loss: 0.6208498477935791
Loss: 0.8903772234916687
Loss: 0.631838858127594
Loss: 0.5726516246795654
Loss: 0.9745912551879883
Loss: 0.7749619483947754
Loss: 1.2348846197128296
Loss: 1.4581211805343628
Loss: 1.0184781551361084
Loss: 1.1735872030258179
Loss: 0.6742292642593384
Loss: 0.81309974193573
Loss: 0.5625093579292297
Loss: 0.8396608233451843
Loss: 0.6596705913543701
Loss: 0.9394886493682861
Loss: 0.7282381057739258
Loss: 0.6650823950767517
Loss: 0.4493129253387451
Loss: 0.803769052028656
Loss: 0.9731718897819519
Loss: 0.8521102666854858
Loss: 1.0228859186172485
Loss: 0.7859153747558594
Loss: 0.7056289911270142
Loss: 1.0703493356704712
Loss:

Loss: 0.5439621210098267
Loss: 1.3448187112808228
Loss: 0.7098813056945801
Loss: 0.5693413019180298
Loss: 0.9082804322242737
Loss: 0.8474861979484558
Loss: 0.71600741147995
Loss: 1.0205072164535522
Loss: 0.9379904866218567
Loss: 0.501567542552948
Loss: 0.801571786403656
Loss: 0.9475910663604736
Loss: 0.7214249968528748
Loss: 0.5480141043663025
Loss: 1.1273220777511597
Loss: 0.7417948842048645
Loss: 0.6312499046325684
Loss: 0.690985381603241
Loss: 0.822965145111084
Loss: 0.5555624961853027
Loss: 0.5432143211364746
Loss: 0.9471821188926697
Loss: 0.38785359263420105
Loss: 0.8171126246452332
Loss: 0.5570372939109802
Loss: 0.6687296628952026
Loss: 0.7531300187110901
Loss: 0.5471327304840088
Loss: 1.1817951202392578
Loss: 0.4625872075557709
Loss: 0.4688161015510559
Loss: 0.7400789260864258
Loss: 0.8815799355506897
Loss: 0.5065590143203735
Loss: 0.8422237634658813
Loss: 1.0513054132461548
Loss: 0.30958089232444763
Loss: 0.5879249572753906
Loss: 1.3142390251159668
Loss: 0.8865683674812317
Loss

Loss: 0.7311410307884216
Loss: 0.9891135096549988
Loss: 1.0580891370773315
Loss: 1.0131064653396606
Loss: 0.5156431198120117
Loss: 0.5804809927940369
Loss: 0.8831046223640442
Loss: 0.5333790183067322
Loss: 0.9904941916465759
Loss: 1.7144702672958374
Loss: 0.7138749957084656
Loss: 0.36085131764411926
Loss: 0.642808198928833
Loss: 0.9326474070549011
Loss: 1.3026983737945557
Loss: 0.9185518622398376
Loss: 0.41406092047691345
Loss: 0.941785454750061
Loss: 1.4239767789840698
Loss: 0.8702871203422546
Loss: 0.597818911075592
Loss: 0.5524634718894958
Loss: 1.352586269378662
Loss: 0.2130550593137741
Loss: 0.913188636302948
Loss: 0.3763737082481384
Loss: 0.5873268246650696
Loss: 0.8015717267990112
Loss: 1.029675006866455
Loss: 0.6915457248687744
Loss: 0.23499278724193573
Loss: 0.903390645980835
Loss: 0.49600502848625183
Loss: 0.8171234726905823
Loss: 0.4568544924259186
Loss: 0.81952303647995
Loss: 1.0539506673812866
Loss: 0.7562544941902161
Loss: 0.5315084457397461
Loss: 0.5801301598548889
Loss:

Loss: 0.5298357605934143
Loss: 0.8801218867301941
Loss: 0.9320566654205322
Loss: 0.9411020278930664
Loss: 0.7190672755241394
Loss: 0.3736785054206848
Loss: 0.8851849436759949
Loss: 0.4726160764694214
Loss: 0.44180458784103394
Loss: 0.7505762577056885
Loss: 0.44348251819610596
Loss: 1.1156575679779053
Loss: 0.35152921080589294
Loss: 0.7879107594490051
Loss: 0.5007035136222839
Loss: 0.7526969313621521
Loss: 0.5942320823669434
Loss: 0.9575000405311584
Loss: 0.7679378390312195
Loss: 0.7354762554168701
Loss: 0.42532745003700256
Loss: 0.7542977929115295
Loss: 0.5900890231132507
Loss: 1.494094729423523
Loss: 0.6778708100318909
Loss: 0.9068226218223572
Loss: 0.6049314141273499
Loss: 0.7228426337242126
Loss: 0.734055757522583
Loss: 1.270025610923767
Loss: 0.5996488928794861
Loss: 1.100575566291809
Loss: 1.0289082527160645
Loss: 0.9568344950675964
Loss: 0.8533728122711182
Loss: 0.6369470953941345
Loss: 1.0829224586486816
Loss: 0.9249605536460876
Loss: 0.9308835864067078
Loss: 0.3559556007385254


Loss: 0.5400364995002747
Loss: 0.9848331809043884
Loss: 0.3362034261226654
Loss: 0.7727022767066956
Loss: 0.23743267357349396
Loss: 0.6512264609336853
Loss: 0.7110605835914612
Loss: 1.9500069618225098
Loss: 0.5883908867835999
Loss: 0.9760526418685913
Loss: 0.6223357319831848
Loss: 1.0392848253250122
Loss: 1.0942976474761963
Loss: 0.5667153000831604
Loss: 0.46768563985824585
Loss: 1.025577187538147
Loss: 0.871694028377533
Loss: 0.3826308846473694
Loss: 0.729840099811554
Loss: 0.4593898355960846
Loss: 0.814139723777771
Loss: 0.8302038908004761
Loss: 0.8555635809898376
Loss: 0.6065970659255981
Loss: 0.5658639073371887
Loss: 0.415926456451416
Loss: 0.7353091239929199
Loss: 0.9644647836685181
Loss: 0.8448164463043213
Loss: 0.636635959148407
Loss: 0.3959242105484009
Loss: 1.094330072402954
Loss: 0.9095768332481384
Loss: 0.5117197632789612
Loss: 0.5330187082290649
Loss: 0.5495163202285767
Loss: 1.3196133375167847
Loss: 1.2294299602508545
Loss: 0.7597182989120483
Loss: 0.9759388566017151
Loss:

In [29]:
from huggingface_hub import notebook_login

notebook_login()

model.save_pretrained('bert-base-paradetox-editOps-editDistance')
tokenizer.save_pretrained('bert-base-paradetox-editOps-editDistance')

model.push_to_hub('bert-base-paradetox-editOps-editDistance')
tokenizer.push_to_hub('bert-base-paradetox-editOps-editDistance')